# EC2202 Queues

**Disclaimer.**
This code examples are based on 

1. [KAIST CS206 (Professor Otfried Cheong)](https://otfried.org/courses/cs206/)
2. [LeetCode](https://leetcode.com/)
3. [GeeksForGeeks](https://practice.geeksforgeeks.org/)
4. Coding Interviews

### Evaluate Reverse Polish Notation

Reverse Polish notation, also referred to as Polish postfix notation is a way of laying out operators and operands.

When making mathematical expressions, we typically put arithmetic operators (like +, -, *, and /) between operands. For example: `(3 + 1) * 4 = 16`

However, in Reverse Polish Notation, the operators come after the operands. For example: `3 1 + 4 *`

#### 'ppp' Exercise

In [ ]:
import doctest

def eval_rpn(tokens):
  '''
  >>> eval_rpn(["3", "1", "+", "4", "*"])  # (3 + 1) * 4 = 16
  16
  >>> eval_rpn(["2","1","+","3","*"])      # ((2 + 1) * 3) = 9
  9
  >>> eval_rpn(["4","13","5","/","+"])     # (4 + (13 / 5)) = 6
  6
  >>> eval_rpn(["10","6","9","3","+","-11","*","/","*","17","+","5","+"])  # ((10 * (6 / ((9 + 3) * -11))) + 17) + 5
  22
  '''
  # from class
  stack = []
  for letter in tokens:
    if letter in '+-*/':
      oper1 = stack.pop()
      oper2 = stack.pop()
      if letter == '+':
        res = oper1 + oper2
      if letter == '-':
        res = oper2 - oper1
      if letter == '*':
        res = oper2 * oper1
      if letter == '/':
        res = int(oper2 / oper1)
      stack.append(res)
    else:
      stack.append(int(letter))
  return stack[0]

  # operations = {
  #   "+": lambda a, b: a + b,
  #   "-": lambda a, b: a - b,
  #   "/": lambda a, b: int(a/b),
  #   "*": lambda a, b: a * b
  # }

  # stack = Stack()
  
  # for tok in tokens:
  #   if tok in operations:
  #     n2 = stack.pop()
  #     n1 = stack.pop()
  #     stack.push(operations[tok](n1, n2))
  #   else:
  #     stack.push(int(tok))
  # return stack.pop()

In [ ]:
doctest.run_docstring_examples(eval_rpn, globals(), False, __name__)

In [ ]:
class EmptyQueueError(Exception):
  pass

class FullQueueError(Exception):
  pass

## Implementing a Queue

### Using the Python List

#### Basic Idea

The Python list makes our life simple.

In [ ]:
class Queue():
  def __init__(self):
    self._data = []
    self._size = 0

  def is_empty(self):
    return len(self._data) == 0  # return self._size == 0
  
  def front(self):
    if self.is_empty():
      raise EmptyQueueError
    return self._data[0]

  def dequeue(self):  # takes O(N) :<
    if self.is_empty():
      raise EmptyQueueError
    self._size -= 1
    return self._data.pop(0)  # takes O(N)

  def enqueue(self, x):
    self._data.append(x)
    self._size += 1
  
  def size(self):
    return self._size

In [ ]:
queue = Queue()
print('after we initialized a queue:', queue.is_empty())
queue.enqueue(1)
print('after pushing 1 into the queue:', queue.is_empty())
queue.enqueue(2)
queue.enqueue(3)
print('the front of the queue:', queue.front())
print('dequeue from the queue:', queue.dequeue())
print('dequeue from the queue:', queue.dequeue())
print('dequeue from the queue:', queue.dequeue())
print('after dequeue three times from the queue:', queue.is_empty())

after we initialized a queue: True
after pushing 1 into the queue: False
the front of the queue: 1
dequeue from the queue: 1
dequeue from the queue: 2
dequeue from the queue: 3
after dequeue three times from the queue: True


#### More Efficient Implementation

We are going to implement a Queue with two Python lists that takes O(1) on average for all operations?

In [ ]:
class Queue():
  def __init__(self):
    self._in = []
    self._out = []

  def is_empty(self):
    return len(self._in) == 0 and len(self._out) == 0
  
  def _sort_out(self):
    if self.is_empty():
      raise EmptyQueueError
    if len(self._out) == 0:
      self._out = self._in
      self._in = []
      self._out.reverse()

  def front(self):
    self._sort_out()
    return self._out[-1]

  def dequeue(self):
    self._sort_out()
    return self._out.pop()  # takes O(1) on average, worst-case: O(N)

  def enqueue(self, x):
    self._in.append(x)

In [ ]:
queue = Queue()
print('after we initialized a queue:', queue.is_empty())
queue.enqueue(1)
print('after pushing 1 into the queue:', queue.is_empty())
queue.enqueue(2)
queue.enqueue(3)
print('the front of the queue:', queue.front())
print('dequeue from the queue:', queue.dequeue())
print('dequeue from the queue:', queue.dequeue())
print('dequeue from the queue:', queue.dequeue())
print('after dequeue three times from the queue:', queue.is_empty())

after we initialized a queue: True
after pushing 1 into the queue: False
the front of the queue: 1
dequeue from the queue: 1
dequeue from the queue: 2
dequeue from the queue: 3
after dequeue three times from the queue: True


### Using Arrays

However, you might not be able to use Python all the times. You sometimes need to write code in other languages such as C, C++ or Java. In such cases, we implement a Queue with arrays. Furthermore, we implement a Queue as a **circular Queue**.

#### 'ppp' Exercise

Can you implement a circular Queue using arrays?

In [ ]:
class Queue():
  # 1) Initialized: 5 => 4 items
  #        [ None, None, None, None, None ]
  # front     ^  
  # rear      ^

  # 2) Add one item
  #        [ Item, None, None, None, None ]
  # front     ^  
  # rear            ^

  # 3) Add some items
  #        [ None, Item, Item, None, None ]
  # front           ^  
  # rear                        ^

  # 4) Add more items
  #        [ Item, None, None, Item, Item ]
  # front                       ^  
  # rear            ^

  # 5) Add even more items
  #        [ Item, Item, Item, Item, Item ]
  # front                       ^  
  # rear                        ^

  # What does front == rear mean? Full buffer or empty buffer?
  # Typical solution: Forbid filling buffer completely
  #                   always keep one slot free.

  def __init__(self, capacity):
    self._data = [ None ] * capacity
    self._front = 0
    self._rear = 0

  def is_empty(self):
    return self._front == self._rear
  
  # we always keep one slot empty to distinguish full from empty
  def is_full(self):
    # return self._rear + 1 == self._front
    return ((self._rear + 1) % len(self._data)) == self._front
  
  def front(self):
    if self.is_empty():
      raise EmptyQueueError
    return self._data[self._front]

  def dequeue(self):
    item = self.front()
    self._front = (self._front + 1) % len(self._data)
    return item

  def enqueue(self, item):
    if self.is_full():
      raise FullQueueError
    self._data[self._rear] = item
    self._rear = (self._rear + 1) % len(self._data)

In [ ]:
queue = Queue(5)
print('after we initialized a queue:', queue.is_empty())
queue.enqueue(1)
print('after pushing 1 into the queue:', queue.is_empty())
queue.enqueue(2)
queue.enqueue(3)
print('the front of the queue:', queue.front())
print('dequeue from the queue:', queue.dequeue())
print('dequeue from the queue:', queue.dequeue())
print('dequeue from the queue:', queue.dequeue())
print('after dequeue three times from the queue:', queue.is_empty())

# extra test cases for array-based queue
# queue.enqueue(1)
# queue.enqueue(2)
# queue.enqueue(3)
# queue.enqueue(4)
# queue.enqueue(5)  # full-queue error occurs
# queue.enqueue(6)
# queue.enqueue(7)
queue.enqueue(1)
print("queue._front & queue._rear", queue._front, queue._rear)
queue.enqueue(2)
print("queue._front & queue._rear", queue._front, queue._rear)
queue.enqueue(3)
print("queue._front & queue._rear", queue._front, queue._rear)
queue.enqueue(4)
print("queue._front & queue._rear", queue._front, queue._rear)
queue.dequeue()
print("queue._front & queue._rear", queue._front, queue._rear)
queue.dequeue()
print("queue._front & queue._rear", queue._front, queue._rear)
queue.enqueue(1)
print("queue._front & queue._rear", queue._front, queue._rear)
queue.enqueue(2)
print("queue._front & queue._rear", queue._front, queue._rear)
queue.dequeue()
print("queue._front & queue._rear", queue._front, queue._rear)
queue.dequeue()
print("queue._front & queue._rear", queue._front, queue._rear)
queue.dequeue()
print("queue._front & queue._rear", queue._front, queue._rear)
queue.enqueue(3)
print("queue._front & queue._rear", queue._front, queue._rear)
queue.enqueue(4)
print("queue._front & queue._rear", queue._front, queue._rear)

after we initialized a queue: True
after pushing 1 into the queue: False
the front of the queue: 1
dequeue from the queue: 1
dequeue from the queue: 2
dequeue from the queue: 3
after dequeue three times from the queue: True
queue._front & queue._rear 3 4
queue._front & queue._rear 3 0
queue._front & queue._rear 3 1
queue._front & queue._rear 3 2
queue._front & queue._rear 4 2
queue._front & queue._rear 0 2
queue._front & queue._rear 0 3
queue._front & queue._rear 0 4
queue._front & queue._rear 1 4
queue._front & queue._rear 2 4
queue._front & queue._rear 3 4
queue._front & queue._rear 3 0
queue._front & queue._rear 3 1


### Using Linked Nodes

As we made our list efficient by linking nodes (linked lists), we could do the similar thing for array Queues to make them efficient. 

#### 'ppp' Exercise

Implement Stack using the `_Node` class

In [ ]:
class _Node():
  def __init__(self, item, next=None):
    self.item = item
    self.next = next
 
class Queue:
  def __init__(self):
    self._front = None
    self._rear = None
 
  def is_empty(self):
    return self._front == None
  
  def front(self):
    if self.is_empty():
      return
    return self._front.item
  
  # F -> N1 -> N2
  #      N1 -> N2
  def dequeue(self):    
    if self.is_empty():
      raise EmptyQueueError
    temp = self._front.item
    self._front = self._front.next

    if(self._front == None):
      self._rear = None
    return temp
    # garbage collection
    # an object; if nothing is pointing to that object (there is no name)
    # => this object together with the memory space gets deleted automatically
  
  def enqueue(self, item):
    temp = _Node(item)

    #     Node(3) -> Node(4) -> Node(7)
    # f:    ^
    # r:                          ^
    if self.is_empty():
      self._front = temp
      self._rear = temp
    else:
      self._rear.next = temp
      self._rear = temp  # self._rear.next

In [ ]:
queue = Queue()
print('after we initialized a queue:', queue.is_empty())
queue.enqueue(1)
print('after pushing 1 into the queue:', queue.is_empty())
queue.enqueue(2)
queue.enqueue(3)
print('the front of the queue:', queue.front())
print('dequeue from the queue:', queue.dequeue())
print('dequeue from the queue:', queue.dequeue())
print('dequeue from the queue:', queue.dequeue())
print('after dequeue three times from the queue:', queue.is_empty())

after we initialized a queue: True
after pushing 1 into the queue: False
the front of the queue: 1
dequeue from the queue: 1
dequeue from the queue: 2
dequeue from the queue: 3
after dequeue three times from the queue: True


## Applications of Queues

#### [Facebook] Open the Locks

You have a lock in front of you with 4 circular wheels. Each wheel has 10 slots: '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'. The wheels can rotate freely and wrap around: for example we can turn '9' to be '0', or '0' to be '9'. Each move consists of turning one wheel one slot.

The lock initially starts at '0000', a string representing the state of the 4 wheels.

You are given a list of `deadends` dead ends, meaning if the lock displays any of these codes, the wheels of the lock will stop turning and you will be unable to open it.

Given a `target` representing the value of the wheels that will unlock the lock, return the minimum total number of turns required to open the lock, or -1 if it is impossible.

##### 'ppp' Exercise

In [ ]:
def open_lock(deadends, target):
  '''
  >>> open_lock(["0201","0101","0102","1212","2002"], "0202")
  6
  >>> open_lock(["8888"], "0009")
  1
  >>> open_lock(["8887","8889","8878","8898","8788","8988","7888","9888"], "8888")
  -1
  '''
  #    '0000'
  # -> '1000'
  # -> '9000'
  # -> '0100'
  # -> '0900'
  queue = Queue()
  queue.enqueue(("0000", 0))
  visited = ['0000']  # set('0000')
  
  while queue.size() > 0:
    string, step = queue.dequeue()
    
    if string in deadends:
      continue
    if string == target:
      return step
    for i in range(4):
      num = int(string[i])
      for dx in (-1, 1):
        num_new = (num + dx) % 10  # 0 -> 9, 1 & -1 % 10 = 9
        string_new = string[:i] + str(num_new) + string[i+1:]

        if string_new not in visited:
          queue.enqueue((string_new, step+1))
          visited.append(string_new)  # visited.add(string_new)
  return -1

  # Breath-first search

In [ ]:
doctest.run_docstring_examples(open_lock, globals(), False, __name__)